# Scraping NYT Articles and using NLP methods 
This is to identify and learn about potential EdTech donors!  

In [14]:
# might need to use Beautiful Soup and Pickle for this work too.
import requests
import os
from pprint import pprint

#my usual suspects
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
%matplotlib inline

In [60]:
nyt_apikey = open('../nyt_apikey.txt','r')

In [61]:
file = nyt_apikey.read().splitlines()

In [62]:
myapikey = file[0]

In [63]:
apikey = os.getenv('NYTIMES_APIKEY', myapikey)

## Look up for current top stories:
https://developer.nytimes.com/docs/top-stories-product/1/overview

In [64]:
section = "education"
query_url = f"https://api.nytimes.com/svc/topstories/v2/{section}.json?api-key={apikey}"

r = requests.get(query_url)
#pprint(r.json())

## Article Search Endpoint
https://developer.nytimes.com/docs/articlesearch-product/1/routes/articlesearch.json/get

In [53]:
query = "education"
begin_date = "20210925" # YYYYMMDD
#filter_query = "\"body:(\"space\") AND glocations:(\"BOSTON\")\""
#filter_query = "\"news_desk:(\"education\") AND body:(\"fund\") AND glocations:(\"NEW YORK CITY\")\""
filter_query = "\"news_desk: (\"education\") AND body:(\"fundraising\") \""
page = "0" # <0-100>
sort = "relevance" # newest, oldest

query_url = f"https://api.nytimes.com/svc/search/v2/articlesearch.json?" \
f"q={query}" \
f"&api-key={apikey}" \
f"&begin_date={begin_date}" \
f"&fq={filter_query}" \
f"&page={page}" \
f"&sort={sort}" \

r = requests.get(query_url)

jsonfile = r.json()
#pprint(jsonfile)

In [54]:
jsonfile.keys()

dict_keys(['status', 'copyright', 'response'])

In [55]:
len(jsonfile['response']['docs'])

10

In [69]:
articles = jsonfile['response']['docs']
articles[0].keys()

dict_keys(['abstract', 'web_url', 'snippet', 'lead_paragraph', 'source', 'multimedia', 'headline', 'keywords', 'pub_date', 'document_type', 'news_desk', 'section_name', 'byline', 'type_of_material', '_id', 'word_count', 'uri'])

In [100]:
data = {'abstract': [],
        'web_url': [],
        'lead_paragraph': [],
        'headline': [],  
        'date': [], 
        'doc_type': [],
        'material_type': [],
        'keywords': []}

In [94]:
import dateutil
from dateutil.relativedelta import relativedelta

In [101]:
for article in articles:
    data['abstract'].append(article['abstract'])
    data['web_url'].append(article['web_url'])
    data['lead_paragraph'].append(article['lead_paragraph'])
    data['headline'].append(article['headline']['main'])
    data['date'].append(article['pub_date'])
    data['doc_type'].append(article['document_type'])
    data['material_type'].append(article['type_of_material'])
    
    keywords = [keyword['value'] for keyword in article['keywords'] if keyword['name'] == 'subject']
    data['keywords'].append(keywords)

In [102]:
df = pd.DataFrame(data) 

In [104]:
df.head(2)

,abstract,web_url,lead_paragraph,headline,date,doc_type,material_type,keywords
0,Following a collection of vignettes from schoo...,https://www.nytimes.com/2021/09/24/education/s...,"Last week, we published a special feature on s...",‘I’m So Tired’: Readers Respond to Schools Reo...,2021-09-24T09:00:25+0000,article,News,"[Education (K-12), Coronavirus Risks and Safet..."
1,Some districts have established robust virus t...,https://www.nytimes.com/2021/09/25/health/scho...,"SAN ANTONIO — One recent Thursday, Ciara Brown...","‘I Need an Army’: Across America, Schools Cram...",2021-09-25T13:00:13+0000,article,News,"[your-feed-science, Tests (Medical), Education..."


In [10]:
def get_articles(file):
    
    article_results = []
    
    for i in range(len(file)):
        article_dict = {}
        
        article_dict['web_url'] = file[i]['web_url']
        article_dict['abstract'] = file[i]['abstract']
        
        article_results.append(article_dict)
        
    return article_results

In [11]:
list_articles = get_articles(jsonfile['response']['docs'])

In [13]:
list_articles[0]

{'web_url': 'https://www.nytimes.com/2021/09/24/education/schools-reopening-comments.html',
 'abstract': 'Following a collection of vignettes from schools around the country, readers weighed in. Here is a selection of their comments.'}